DL - https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

In [137]:
import os
import pandas as pd 
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize as tokenize
from nltk.corpus import stopwords as stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torch import nn
from tqdm import tqdm
import uuid

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rd81\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rd81\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rd81\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
try :
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False
print('IN COLAB: ', IN_COLAB)


parentdir = "./propaganda_dataset_v2"
train_file= "propaganda_train.tsv"
val_file= "propaganda_val.tsv"

train_path=os.path.join(parentdir,train_file)
val_path=os.path.join(parentdir,val_file)

if IN_COLAB:
  train_path = '/content/propaganda_train.tsv'
  val_path = '/content/propaganda_val.tsv'

train_df=pd.read_csv(train_path,delimiter="\t",quotechar='|')
val_df=pd.read_csv(val_path,delimiter="\t",quotechar='|')


IN COLAB:  False


In [9]:
def transform_binaryify(row):
    new_value = 0 if row['label'] == 'not_propaganda' else 1
    return new_value

def transform_strip_tag(row):
    sent = row['tagged_in_context']
    cleaned_string = sent.replace("<BOS>", "")
    cleaned_string = cleaned_string.replace("<EOS>", "")
    return cleaned_string


train_df['propaganda'] = train_df.apply(transform_binaryify, axis=1)
train_df['original_without_snip_tags'] = train_df.apply(transform_strip_tag, axis=1)


val_df['propaganda'] = val_df.apply(transform_binaryify, axis=1)
val_df['original_without_snip_tags'] = val_df.apply(transform_strip_tag, axis=1)

train_df
val_df

,label,tagged_in_context,propaganda,original_without_snip_tags
0,not_propaganda,"On average, between 300 and 600 infections are...",0,"On average, between 300 and 600 infections are..."
1,causal_oversimplification,Mostly because <BOS> the country would not las...,1,Mostly because the country would not last lon...
2,appeal_to_fear_prejudice,Lyndon Johnson <BOS> gets Earl Warren and Sen....,1,Lyndon Johnson gets Earl Warren and Sen. Rich...
3,not_propaganda,<BOS> You <EOS> may opt out at anytime.,0,You may opt out at anytime.
4,repetition,It must be exacted from him directly in order ...,1,It must be exacted from him directly in order ...
...,...,...,...,...
635,not_propaganda,"NewsCatholic Church, <BOS> Family, Marriage <E...",0,"NewsCatholic Church, Family, Marriage"
636,not_propaganda,"Remember our saying, modern day fairy <BOS> ta...",0,"Remember our saying, modern day fairy tales s..."
637,not_propaganda,Why <BOS> not <EOS> open up to Iran with massi...,0,Why not open up to Iran with massive amounts...
638,flag_waving,<BOS> He also sang an Islamic State fight song...,1,He also sang an Islamic State fight song and ...


In [5]:

#replace with the path where you have downloaded your model.
pretrained_model_path = './GoogleNews-vectors-negative300.bin.gz' 
#initialise the pre trained model using load_word2vec_format from gensim module.
my_model = KeyedVectors.load_word2vec_format(pretrained_model_path, binary=True)
 

In [99]:
def transform_tokens_to_w2v_with_stems(string, model, pool_type='mean', normalise = False):
    stops = set(stopwords.words('english'))
    porter_stemmer = PorterStemmer()
    snowball_stemmer = SnowballStemmer('english')
    vectors = []
    original_tokens = tokenize(string)
    cleaned_tokens = [w for w in original_tokens if len(w) > 1 and w.isalpha() and w not in stops]
    
    for token in cleaned_tokens:
        if token in model.key_to_index:
            vectors.append(model.get_vector(token, norm=normalise))
        else:
            porter_stem = porter_stemmer.stem(token)
            snowball_stem = snowball_stemmer.stem(token)
            if porter_stem in model.key_to_index:
                vectors.append(model.get_vector(porter_stem, norm=normalise))
            elif snowball_stem in model.key_to_index:
                vectors.append(model.get_vector(snowball_stem, norm=normalise))
    
    if not vectors:
        return np.zeros(300)
    
    if pool_type == 'mean':
        return np.mean(vectors, axis=0)
    elif pool_type == 'max':
        return np.max(vectors, axis=0)
    else:
        raise ValueError("Invalid pool_type. Choose either 'mean' or 'max'.")


def transform_tokens_to_w2v_no_stems(string, model, pool_type='mean', normalise = False):
    vectors = []
    stops = set(stopwords.words('english'))
    original_tokens = tokenize(string)
    cleaned_tokens = [w for w in original_tokens if len(w) > 1 and w.isalpha() and w not in stops]
    
    for token in cleaned_tokens:
        if token in model.key_to_index:
            vectors.append(model.get_vector(token, norm=normalise))
    
    if not vectors:
        return np.zeros(300)
    
    if pool_type == 'mean':
        return np.mean(vectors, axis=0)
    elif pool_type == 'max':
        return np.max(vectors, axis=0)
    else:
        raise ValueError("Invalid pool_type. Choose either 'mean' or 'max'.")

In [100]:
class CustomPropagandaDataset_vanilla(Dataset):
    def __init__(self,df, model, stems=False):
        self.stems = stems
        self.labels=torch.tensor([label for label in df['propaganda']])
        
        if self.stems==True:
            self.texts=torch.tensor([transform_tokens_to_w2v_with_stems(string, model, normalise = False) for string in df['original_without_snip_tags']])
        else:
            self.texts=torch.tensor([transform_tokens_to_w2v_no_stems(string, model, normalise = False) for string in df['original_without_snip_tags']])
        
    def classes(self):
        return self.labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        texts=self.texts[idx]
        y=self.labels[idx]
        return texts,y


def prepare_inputs(input1,label,device):
  label=label.to(device)
  mask=input1['attention_mask'].to(device)
  input_id=input1['input_ids'].squeeze(1).to(device)
  return (input_id,mask,label)

In [140]:

batch_size=50


In [141]:
train_data = CustomPropagandaDataset_vanilla(train_df, my_model, stems=False)
val_data = CustomPropagandaDataset_vanilla(val_df, my_model, stems=False)

In [142]:
train_dataloader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
val_dataloader=torch.utils.data.DataLoader(val_data,batch_size=batch_size)

In [113]:
for i in train_dataloader:
    print(i)
    break

[tensor([[-0.1965,  0.1516,  0.2344,  ..., -0.0049,  0.0892, -0.0312],
        [ 0.1219,  0.0512,  0.0539,  ..., -0.0454,  0.0167, -0.0031],
        [ 0.0011,  0.0667, -0.0239,  ...,  0.0096,  0.0224, -0.0241],
        ...,
        [ 0.0986,  0.0476,  0.0466,  ..., -0.0600,  0.0675, -0.0768],
        [ 0.0506,  0.0747, -0.0983,  ..., -0.1090,  0.0623, -0.0176],
        [-0.0213,  0.0306, -0.0751,  ...,  0.0904,  0.0625,  0.0396]],
       dtype=torch.float64), tensor([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0])]


In [ ]:
# try 
# stems yes no
# norm yes no
# pool mean max



In [163]:

class Custom_MLP(nn.Module):

    def __init__(self,dropout=0.5,num_classes=2):
        super().__init__()
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(dropout)

        self.linear_1=nn.Linear(300, 128)
        self.linear_2=nn.Linear(128, 128)
        self.linear_3=nn.Linear(128,num_classes)

    def forward(self,inputs):
        
        x = self.linear_1(inputs)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.linear_2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.linear_3(x)
        
        return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [177]:
epochs =100
lr = 5e-5
n_classes = 2

In [178]:
model=Custom_MLP(num_classes=n_classes).to(device)
criterion=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

In [179]:
train_acc_list = []
train_loss_list = []
val_acc_list = []
val_loss_list = []

best_val_acc = 0
best_epoch = 0
best_model_state = None

model_id = str(uuid.uuid4())


for epoch_num in range(epochs):

    total_acc_train=0
    total_loss_train=0
    model.train()

    for train_input,train_label in train_dataloader:

        train_input = train_input.to(device).float()
        train_label = train_label.to(device)
        output_1=model(train_input)

        batch_loss_1=criterion(output_1,train_label)

        total_loss_train += batch_loss_1.item()

        acc=(output_1.argmax(dim=1)==train_label).sum().item()
        total_acc_train+=acc

        model.zero_grad()
        batch_loss_1.backward()
        optimizer.step()

    total_acc_val=0
    total_loss_val=0

    y_true = []
    y_pred = []
    model.eval()
    with torch.no_grad():
        for val_input,val_label in val_dataloader:

            val_input = val_input.to(device).float()
            val_label = val_label.to(device)

            output_2=model(val_input)

            batch_loss_2=criterion(output_2,val_label)

            total_loss_val+=batch_loss_2.item()

            acc=(output_2.argmax(dim=1)==val_label).sum().item()
            total_acc_val+=acc

            # for scoring
            predicted = output_2.argmax(dim=1)
            y_true.extend(val_label.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    train_acc = total_acc_train / len(train_data)
    train_loss = total_loss_train / len(train_data)
    val_acc = total_acc_val / len(val_data)
    val_loss = total_loss_val / len(val_data)

    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)

    # print(f'Epochs: {epoch_num+1} | Train Loss: {total_loss_train / len(train_data):.3f} | Train Accuracy: {total_acc_train/len(train_data):.3f}')
    # print(f'Val loss: {total_loss_val/len(val_data):.3f} | Val Accuracy: {total_acc_val / len(val_data):.3f}')
    if val_acc > best_val_acc:
        best_epoch = epoch_num
        y_true_best = y_true.copy()
        y_pred_best = y_pred.copy()
        best_val_acc = val_acc
        best_model_state = model.state_dict()

    # if epoch_num == epochs-1:
    #     print(f'______{model_id}______')
    #     print(f'LR: {lr} FINAL ACC = {total_acc_val / len(val_data):.3f}')
    #     print(f'LR: {lr} BEST ACC = {best_val_acc:.3f}')
    #     print('____________')
        
# Plot the accuracy and loss curves over epochs
epochs_range = range(1, epochs+1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_acc_list, label='Training Accuracy')
plt.plot(epochs_range, val_acc_list, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy Curves')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_loss_list, label='Training Loss')
plt.plot(epochs_range, val_loss_list, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Loss Curves')

plt.tight_layout()
plt.show()
# analysis of best performing model

RuntimeError: In FT2Font: Can not load face (unknown file format; error code 0x2)

Error in callback <function _draw_all_if_interactive at 0x000002053A9405E0> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: In FT2Font: Can not load face (unknown file format; error code 0x2)

RuntimeError: In FT2Font: Can not load face (unknown file format; error code 0x2)

<Figure size 1200x600 with 2 Axes>